### 检测程序效率

**cv2.getTickCount** -------> 参考点到该时刻的时钟数  
在一个函数体前后调用，返回函数执行时间， 类似 tic, toc.  
**cv2.getTickFrequency** ---------> 时钟频率  hz
``` python
e1 = cv2.getTickCount()
# your code execution
e2 = cv2.getTickCount()
time = (e2 - e1)/ cv2.getTickFrequency()
```

In [1]:
import cv2
import numpy as np
import copy as cp
from matplotlib import pyplot as plt

In [2]:
help(cv2.medianBlur)

Help on built-in function medianBlur:

medianBlur(...)
    medianBlur(src, ksize[, dst]) -> dst
    .   @brief Blurs an image using the median filter.
    .   
    .   The function smoothes an image using the median filter with the \f$\texttt{ksize} \times
    .   \texttt{ksize}\f$ aperture. Each channel of a multi-channel image is processed independently.
    .   In-place operation is supported.
    .   
    .   @note The median filter uses #BORDER_REPLICATE internally to cope with border pixels, see #BorderTypes
    .   
    .   @param src input 1-, 3-, or 4-channel image; when ksize is 3 or 5, the image depth should be
    .   CV_8U, CV_16U, or CV_32F, for larger aperture sizes, it can only be CV_8U.
    .   @param dst destination array of the same size and type as src.
    .   @param ksize aperture linear size; it must be odd and greater than 1, for example: 3, 5, 7 ...
    .   @sa  bilateralFilter, blur, boxFilter, GaussianBlur



In [9]:
img1=cv2.imread('../images/xxb.jpg')
ima1=cp.deepcopy(img1)
e1=cv2.getTickCount()
for ksize in range(5,49,2):
    ima1=cv2.medianBlur(ima1,ksize)
e2=cv2.getTickCount()
t=(e2-e1)/cv2.getTickFrequency()
print('median blur use {} s'.format(t))


median blur use 3.0556433539563406 s


In [19]:
import time
e1=time.time()  # time.time()直接返回秒数
for ksize in range(5,49,2):
    ima1=cv2.medianBlur(ima1,ksize)
e2=time.time()
t=(e2-e1)
print('median blur use {} s'.format(t))


median blur use 3.0645220279693604 s


## 优化

opencv的优化默认开启

In [30]:

print('优化开启:\t', cv2.useOptimized())
%timeit res = cv2.medianBlur(ima1,49)# 魔法命令， 将该语句执行多次从而得到教准确的执行时间
cv2.setUseOptimized(False)
print('优化开启:\t', cv2.useOptimized())
%timeit res = cv2.medianBlur(ima1,49)
cv2.setUseOptimized(True)

优化开启:	 True
149 ms ± 6.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
优化开启:	 False
150 ms ± 4.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [46]:
x=5
%timeit y=np.square(x**2)
%timeit y=np.square(x)  
print('opencv----------numpy')
%timeit z = cv2.countNonZero(img1[:,:,1])# cv2.countNonZero只能用于单层图像
%timeit z = np.count_nonzero(img1[:,:,1])

738 ns ± 16.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
530 ns ± 13 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
opencv----------numpy
1.48 ms ± 47.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.99 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


<font size="5" color='red'>Python 的标量计算比 Nump 的标量计算要快, np处理多元素数组快</font>  
<font size="5" color='red'>opencv 比np 快， 除了用np 对视图操作</font>

1. 尽量避免使用循环，尤其双层三层循环，它们天生就是非常慢的。
2. 算法中尽量使用向量操作，因为 Numpy 和 OpenCV 都对向量操作进行了优化。
3. 利用高速缓存一致性。
4. 没有必要的话就不要复制数组。使用视图来代替复制。数组复制是非常浪费资源的。  
依旧很慢可以考虑Cython
